__Решение задачи об укладке рюкзака с помощью собственного алгоритма__

Варианты заданий:
* 1.1 случайная генерация
* 2.1 рулетка
* 3.1 с тремя точками
* 4.2 случайное изменение 3х битов у 5% особей
* 5.3 замена своих родителей

In [745]:
import json
import random as rnd

data_path = '26.txt'
result_path = 'res5_2.txt'

__Чтение данных из файла__

In [746]:
def load_data():
    f = open(data_path)        
    first_line = f.readline().split(' ')
    bag_data = {'max_weight': 0, 'max_volume': 0, 'items': []}
    bag_data['max_weight'] = int(first_line[0])
    bag_data['max_volume'] = float(first_line[1])
    
    for line in f:
        values = [x for x in line.split(' ')]
        bag_data['items'].append({'weight': int(values[0]), 'volume': float(values[1]), 'price': int(values[2])})
        
    return bag_data

__Функция приспособления__

In [747]:
def fitness(individual, items):
    weight, volume, price = 0, 0, 0
    for (selected, item) in zip(individual, items):
        if selected:
            weight += item['weight']
            volume += item['volume']
            price += item['price']

    if weight > max_weight or volume > max_volume:
        price = 0

    return price

__Поиск лучшей особи в популяции__

In [748]:
def get_best_individual(population, items):
    candidates = []

    for ind in population:
        candidates.append((fitness(ind, items), ind))

    best_ind = candidates[0]
    for candidate in candidates:
        best_ind = candidate if candidate[0] > best_ind[0] else best_ind

    return best_ind

__Создание популяции__

In [771]:
def create_individual(items):
    individual = [0 for x in range(len(items))]
    
    total_weight = 0
    total_volume = 0
    
    fill_order = [i for i in range(len(items))]
    rnd.shuffle(fill_order)
    
    for i in fill_order:
        item = items[i]
        ind = rnd.randint(0, 1)
        individual[i] = ind
        #если добавление этой вещи в рюкзак делает особь нежизнеспособной - не добавляем
        if total_weight + item['weight'] > max_weight or total_volume + item['volume'] > max_volume:
            individual[i] = 0
            continue
        #если добавили вещь - считаем новый вес и объем    
        else:
            if ind == 1:
                total_weight += item['weight']
                total_volume += item['volume']
    
    return individual

def create_population(items, count):
    return [create_individual(items) for i in range(count)]

__Отбор особей на основе приспособленности__

In [750]:
def choose_to_crossover(population, items):
    choosens = []
    candidates = []

    for i in range(len(population)):
        candidates.append(fitness(population[i], items))
    
    max_fitness = 0
    min_fitness = candidates[0]
    #best_cand = 0
    #worst_cand = 0
    
    #определяем максимальную и минимальную ценность в популяции
    for j in range(len(candidates)):
        if candidates[j] > max_fitness:
            max_fitness = candidates[j]
            #best_cand = j
        if candidates[j] < min_fitness and candidates[j] != 0:
            min_fitness = candidates[j]
            #worst_cand = j

    #print("best", best_cand, 'max min', max_fitness, min_fitness)
    
    #нормализуем значения ценности в отрезок [0, 1] и отбираем родителей
    diff = max_fitness - min_fitness        
    for q in range(len(candidates)):
        candidates[q] = (candidates[q] - min_fitness) / diff
        if rnd.random() < candidates[q]:
            choosens.append(population[q])
    return choosens

__Скрещивание с тремя точками__

In [751]:
def crossover_parents(parent_1, parent_2, items):
    dots = []
    for i in range(0, 3):
        dots.append(rnd.randint(1, len(parent_1)))
    dots.sort()

    child_1 = parent_1[:dots[0]] + parent_2[dots[0]:dots[1]] + parent_1[dots[1]:dots[2]] + parent_2[dots[2]:]
    child_2 = parent_2[:dots[0]] + parent_1[dots[0]:dots[1]] + parent_2[dots[1]:dots[2]] + parent_1[dots[2]:]
    #вместо нежизнеспособного ребенка возвращаем родителя
    if fitness(child_1, items) == 0:
        child_1 = parent_1
    if fitness(child_2, items) == 0:
        child_2 = parent_2

    return child_1, child_2

def crossover(parents, items):
    children = []
    parents_copy = parents.copy()
    #пока в списке больше одного родителя, выбираем двух из них, удаляем из списка и скрещиваем
    while len(parents_copy) > 1:
        parent1 = rnd.choice(parents_copy)
        parents_copy.remove(parent1)
        parent2 = rnd.choice(parents_copy)
        parents_copy.remove(parent2)        
        children.extend(crossover_parents(parent1, parent2, items))
    #добавляем к детям лишнего родителя, если такой был
    children.extend(parents_copy)
    return children
        

__Мутация (изменение 3 вещей у 5% потомков)__

In [758]:
def mutation_individual(individual, items):
    cnt = 0
    while cnt < 3:
        cnt += 1
        ind = rnd.randint(0, len(individual) - 1)
        #инвертируем рандомный бит
        if individual[ind] == 1:
            individual[ind] = 0
        else:
            individual[ind] = 1
            #если особь получилась нежизнеспособной - убираем последнюю вещь и меняем еще один бит
            if fitness(individual, items) == 0:
                individual[ind] = 0
                cnt -= 1


        
def mutation(population, items):    
    len_mutate_pop = int(len(population) * MUTATION_PERC)
    for i in range(len_mutate_pop):
        mutation_individual(population[rnd.randint(0, len(population) - 1)], items)

__Обновление популяции (все родители замещаются их детьми)__

In [753]:
def update_population(population, parents, children):
    new_population = []
    #если особь не была родителем - добавляем ее в новую популяцию
    for individual in population:
        if individual not in parents:
            new_population.append(individual)
    #добавляем всех детей        
    new_population.extend(children)
    return new_population
    
    

__Добавление лучшей особи в популяцию__

In [761]:
def insert_best_individual(population, best_individual, items):
    #если лучшая особь на каком то шаге теряется - добавляем ее в популяцию
    #чтобы популяция не росла - удаляем худшую особь
    for individual in population:
        if best_individual not in population:
            population.append(best_individual)
            worst_individual = best_individual
            worst_price = fitness(worst_individual, items)
            for i in population:
                curr_price = fitness(i, items)
                if curr_price < worst_price:
                    worst_price = curr_price
                    worst_individual = i
            population.remove(worst_individual)
                    
                

__Генетический алгоритм__

In [772]:
def ga(population, items, max_weight, max_volume):
    #массив для хранения последних лучших ценностей
    conv_arr = [0] * CONV_GEN_CNT
    conv_cnt = 0
    #наименее ценная вещь
    cheapest_item = sorted(items, key=lambda dct: dct['price'])[0]['price']
    best_individual = None
    for g in range(CYCLE_CNT):
        
        parents = choose_to_crossover(population, items)

        children = crossover(parents, items)
        
        population = update_population(population, parents, children)
        #так как всё равно дети заменяют родителей, можно сначала заменить, а затем провести мутацию
        #тогда не нужно отдельно мутировать детей и особей, которые не скрещивались
        mutation(population, items)

        #лучшая особь этого поколения
        ind = get_best_individual(population, items)

        #запоминаем лучшую особь среди всех поколений
        if best_individual is None or ind[0] > best_individual[0]:
            best_individual = ind     
        
        #сохраняем лучшую особь, если вдруг мы ее потеряли в процессе скрещивания или мутаций
        insert_best_individual(population, best_individual[1], items)

        conv_arr[conv_cnt] = ind[0]
        conv_cnt += 1
        if conv_cnt >= len(conv_arr):
            conv_cnt = 0
        #если за последние несколько поколений ценность менялась меньше чем на стоимость самой дешевой вещи - заканчиваем
        if abs(min(conv_arr) - max(conv_arr)) <= cheapest_item:
            break
            
    return(best_individual)

__ __

In [770]:
bag_data = load_data()
max_weight = bag_data['max_weight']
max_volume = bag_data['max_volume']
items = bag_data['items']
#количество особей в популяции
fisrt_pop_cnt = 200 
#для скольки поколений оцениваем сходимость
CONV_GEN_CNT = 10
#максимальное количество поколений
CYCLE_CNT = 500
#какая часть поколения мутирует
MUTATION_PERC = 0.05

population = create_population(items, fisrt_pop_cnt)
decision = ga(population, items, max_weight, max_volume)
print(decision)

json_file = open(result_path,'w')
json_file.write("Ценность: " + str(int(decision[0])) + "\nНабор: " + str(decision[1]))
json_file.close()

(4765, [1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0])
